In [10]:
from confluent_kafka import Consumer, KafkaError
import json
from kafka import KafkaConsumer
from datetime import datetime, timedelta
from json import loads

In [19]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
from dash import Dash
import threading
import dash

In [22]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Given data (initialize with empty data)
sbp_data = []
dbp_data = []
timestamps = []

# Define the layout of the app
app.layout = html.Div([
    html.H1("SBP and DBP Plotting App"),

    dcc.Graph(id='sbp-chart'),
    dcc.Graph(id='dbp-chart'),

    dcc.Interval(
        id='interval-component',
        interval=10 * 1000,  # in milliseconds (adjust the interval as needed)
        n_intervals=0
    ),
])

# Define callback to update the charts periodically
@app.callback(
    [Output('sbp-chart', 'figure'),
     Output('dbp-chart', 'figure')],
    [Input('interval-component', 'n_intervals')]
)
def update_charts(n_intervals):
    global sbp_data, dbp_data, timestamps

    # Create a DataFrame for plotting
    df = pd.DataFrame({'Timestamps': pd.to_datetime(timestamps), 'SBP': sbp_data, 'DBP': dbp_data})

    # Create SBP chart
    fig_sbp = {
        'data': [{'x': df['Timestamps'], 'y': df['SBP'], 'type': 'line', 'name': 'SBP'}],
        'layout': {'title': 'SBP Over Time'}
    }

    # Create DBP chart
    fig_dbp = {
        'data': [{'x': df['Timestamps'], 'y': df['DBP'], 'type': 'line', 'name': 'DBP'}],
        'layout': {'title': 'DBP Over Time'}
    }

    return fig_sbp, fig_dbp

# Kafka consumer setup
bootstrap_servers = ['localhost:9092']
input_topic = 'Forecasting_Results_Topic'

def kafka_consumer():
    global sbp_data, dbp_data, timestamps
    consumer = KafkaConsumer(input_topic, bootstrap_servers=bootstrap_servers,
                             auto_offset_reset='earliest', value_deserializer=lambda x: json.loads(x.decode('utf-8')))
    try:
        for message in consumer:
            # Extract features based on your input data
            input_data = message.value

            sbp_predictions = input_data.get("sbp_predictions", [])
            dbp_predictions = input_data.get("dbp_predictions", [])
            timestamps = input_data.get("timestamps", [])

            # Update the Dash app data
            sbp_data.extend(sbp_predictions)
            dbp_data.extend(dbp_predictions)

    except KeyboardInterrupt:
        pass

# Run the Kafka consumer in a separate thread
kafka_thread = threading.Thread(target=kafka_consumer)
kafka_thread.start()

# Run the Dash app in the notebook cell
app.run_server(debug=True, port=8051)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\ASUS\anaconda3\envs\ost_project\Lib\site-packages\dash\dash.py:1255, in Dash.dispatch(self=<dash.dash.Dash object>)
   1254 try:
-> 1255     cb = self.callback_map[output]
        output = 'sbp-chart.figure'
        self.callback_map = {'..sbp-chart.figure...dbp-chart.figure..': {'inputs': [{'id': 'interval-component', 'property': 'n_intervals'}], 'state': [], 'outputs_indices': [0, 1], 'inputs_state_indices': 0, 'long': None, 'output': [<Output `sbp-chart.figure`>, <Output `dbp-chart.figure`>], 'raw_inputs': [<Input `interval-component.n_intervals`>], 'manager': None, 'callback': <function update_charts at 0x000002348B7E6B60>}}
        self = <dash.dash.Dash object at 0x000002349D48AB10>
   1256     func = cb["callback"]

KeyError: 'sbp-chart.figure'

The above exception was the direct cause of the following exception:

K

In [11]:
# Setting up the Python consumer
bootstrap_servers = ['localhost:9092']
# Consumer group ID
group_id = 'group1'
input_topic = 'Forecasting_Results_Topic'

# Create Kafka consumer configuration
consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': group_id,
    'auto.offset.reset': 'earliest'
}


# Your Kafka consumer setup...
consumer = KafkaConsumer(input_topic, bootstrap_servers=['localhost:9093'], auto_offset_reset='earliest',
                             api_version=(0, 10), enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Given data (initialize with empty data)
sbp_data = []
dbp_data = []
timestamps = []

# Define the layout of the app
app.layout = html.Div([
    html.H1("SBP and DBP Plotting App"),

    dcc.Graph(id='sbp-chart'),
    dcc.Graph(id='dbp-chart'),

    dcc.Interval(
        id='interval-component',
        interval=10 * 1000,  # in milliseconds (adjust the interval as needed)
        n_intervals=0
    ),
])

# Define callback to update the charts periodically
@app.callback(
    [Output('sbp-chart', 'figure'),
     Output('dbp-chart', 'figure')],
    [Input('interval-component', 'n_intervals')]
)
def update_charts(n_intervals):
    global sbp_data, dbp_data, timestamps

    # Create a DataFrame for plotting
    df = pd.DataFrame({'Timestamps': timestamps, 'SBP': sbp_data, 'DBP': dbp_data})

    # Create SBP chart
    fig_sbp = {
        'data': [{'x': df['Timestamps'], 'y': df['SBP'], 'type': 'line', 'name': 'SBP'}],
        'layout': {'title': 'SBP Over Time'}
    }

    # Create DBP chart
    fig_dbp = {
        'data': [{'x': df['Timestamps'], 'y': df['DBP'], 'type': 'line', 'name': 'DBP'}],
        'layout': {'title': 'DBP Over Time'}
    }

    return fig_sbp, fig_dbp

In [16]:
try:
    for message in consumer:
        # Extract features based on your input data
        input_data = message.value
        
        received_inputs = [
            input_data.get("sbp_predictions", ""),
            input_data.get("dbp_predictions", ""),
            input_data.get("timestamps", "")
        ]

        SBP = received_inputs[0]
        DBP = received_inputs[1]
        TIMESTAMP = received_inputs[2]
        print(f'SBP: {SBP}')
        print(f'DBP: {DBP}')
        print(f'TIMESTAMP: {TIMESTAMP}')
        print('------------------------------------------------')
except KeyboardInterrupt:
    pass


SBP: [93.33871459960938, 93.41447448730469, 93.32632446289062, 93.1546630859375, 93.05387878417969, 93.04515838623047, 93.08311462402344, 92.8353042602539, 92.97357177734375, 92.89505004882812, 92.71142578125, 92.65310668945312, 92.61247253417969, 92.82862854003906, 92.67825317382812, 92.65972137451172, 92.49693298339844, 92.48892974853516, 92.5070571899414, 92.421875, 92.32555389404297, 92.1645736694336, 92.3563003540039, 92.19361114501953, 92.14399719238281, 92.03890991210938, 92.0396499633789, 92.03141021728516, 91.89105987548828, 91.96728515625]
DBP: [60.424964904785156, 60.299903869628906, 60.26872634887695, 60.20206069946289, 60.19462203979492, 60.05840301513672, 59.993743896484375, 59.973201751708984, 59.936580657958984, 59.94058609008789, 59.90711212158203, 59.90119934082031, 59.874263763427734, 59.84178924560547, 59.759681701660156, 59.75699996948242, 59.66437911987305, 59.64613723754883, 59.66697311401367, 59.623077392578125, 59.62657165527344, 59.622108459472656, 59.63043212